In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow import keras
from numpy.linalg import norm
import os
from tqdm import tqdm
import cv2
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', None)
import warnings   # To avoid warning messages in the code run
warnings.filterwarnings("ignore")

import os # os - operating system (Windows, Mac, Linux) os.mkdir('abc') --> will create a folder for me with the name 'abc'

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
order = list(np.sort(train['label'].unique()))
print(order)

In [ ]:
sns.countplot(train["label"]);

# Basic Model 

In [ ]:
X_train = (train.iloc[:,1:].values).astype('float32') 
y_train = train.iloc[:,0].values.astype('int32') 
X_test = test.values.astype('float32')

X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

X_train = X_train / 255
X_test = X_test / 255

In [ ]:
X_train.shape

In [ ]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=X_train,y=y_train, epochs=20)

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)
predictions

# Model using ResNet50

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
INPUT_SHAPE_RN = (32, 32, 3)

def create_cnn_architecture_model2(input_shape):
    inc_net = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', 
                                                          input_shape=input_shape)
    inc_net.trainable = True
    # Fine-tune the layers
    for layer in inc_net.layers:
        layer.trainable = True
    
    base_inc = inc_net
    base_out = base_inc.output
    pool_out = keras.layers.Flatten()(base_out)
    
    hidden1 = keras.layers.Dense(512, activation='relu')(pool_out)
    drop1 = keras.layers.Dropout(rate=0.3)(hidden1)
    hidden2 = keras.layers.Dense(512, activation='relu')(drop1)
    drop2 = keras.layers.Dropout(rate=0.3)(hidden2)

    out = keras.layers.Dense(10, activation='softmax')(drop2)

    model = keras.Model(inputs=base_inc.input, outputs=out)
    model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-4),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    return model

In [ ]:
model2 = create_cnn_architecture_model2(input_shape=INPUT_SHAPE_RN)
model2.summary()

In [ ]:
y_train = train['label'].values
x_train = train.drop('label', axis = 1).values.reshape((-1, 28, 28, 1)) / 255
# since ResNet is for 3 channel color image but digit is grayscale 1 channle, change it to 3 channel below
x_train = np.concatenate((x_train,x_train,x_train), axis = 3)
print(x_train.shape)

x_test = train.drop('label', axis = 1).values.reshape((-1, 28, 28, 1)) / 255
# since ResNet is for 3 channel color image but digit is grayscale 1 channle, change it to 3 channel below
x_test = np.concatenate((x_test,x_test,x_test), axis = 3)
print(x_test.shape)


In [ ]:
def resize_image_array(img, img_size_dims):
    img = cv2.resize(img, dsize=img_size_dims, 
                     interpolation=cv2.INTER_CUBIC)
    img = np.array(img, dtype=np.float32)
    return img

In [ ]:
IMG_DIMS = (32, 32)

x_train = np.array([resize_image_array(img, img_size_dims=IMG_DIMS) for img in x_train])
x_test = np.array([resize_image_array(img, img_size_dims=IMG_DIMS) for img in x_test])

print('\nTrain_images.shape: {}, of {}'.format(x_train.shape, x_train.dtype))
print('Test_images.shape: {}, of {}'.format(x_test.shape, x_test.dtype))

In [ ]:
EPOCHS = 10
x_train_scaled = x_train / 255.
model2.fit(x_train_scaled, y_train, validation_split=0.1, epochs=EPOCHS)

In [ ]:
x_test_scaled = x_test / 255.
predictions = model2.predict(x_test_scaled)
predictions

In [ ]:

test_id = np.arange(1, x_test.shape[0] + 1, 1)
prediction = np.argmax(predictions, axis = 1)

pred = pd.DataFrame(data = {'ImageId': test_id,
                           'Label': prediction})